In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
# Load the data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# Create our features
X = df.copy()
X.drop(columns=['loan_status'], axis=1, inplace=True)
X = pd.get_dummies(X, drop_first=True)

# Create our target
y = df.loan_status

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(58152, 9)

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9892813049736127

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [17]:
# Train the Logistic Regression model using the resampled data
model1 = LogisticRegression(solver='lbfgs', random_state=1)
model1.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred = model1.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946414201183431

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled2, y_resampled2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
# View the count of target classes with Counter
Counter(y_resampled2)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [22]:
# Train the Logistic Regression model using the resampled data
model2 = LogisticRegression(solver='lbfgs', random_state=1)
model2.fit(X_resampled2, y_resampled2)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [26]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled3, y_resampled3 = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled3)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [27]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=1)
model3.fit(X_resampled3, y_resampled3)

LogisticRegression(random_state=1)

In [28]:
# Calculate the balanced accuracy score
y_pred = model3.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9932813049736127

In [29]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  620,     5],
       [  102, 18657]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



In [31]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled4, y_resampled4 = sm.fit_resample(X_train, y_train)


# View the count of target classes with Counter
Counter(y_resampled4)

Counter({'high_risk': 55636, 'low_risk': 55936})

In [32]:
# Train the Logistic Regression model using the resampled data
model4 = LogisticRegression(solver='lbfgs', random_state=1)
model4.fit(X_resampled4, y_resampled4)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_pred = model4.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [ ]:
#Which model had the best balanced accuracy score?
#Smote Oversampling.
#Which model had the best recall score?
#All models scored similarly.
#Which model had the best geometric mean score?
#All of model had 0.99 scores